In [8]:
import unittest
from mymodule import spyder

In [9]:
class TestSpyder(unittest.TestCase):
    def setUp(self):
        self.base_url = "https://www.bbc.com/news"
        self.links = ["https://www.bbc.com"]
        self.depth = 1
        self.spyder = spyder(self.links, self.base_url, self.depth)
        
    def test_crawl(self):
        visited = self.spyder.crawl(self.base_url, self.depth, 0, set())
        self.assertIn(self.base_url, visited)
            
    def test_check_domain(self):
        links = self.links
        domain_links = self.spyder.check_domain("https://www.bbc.com", ["https://www.bbc.com/news"])
        self.assertIn("https://www.bbc.com/news", domain_links)
        self.assertNotIn("https://www.bbc.co.uk", domain_links)
        
    def test_check_not_domain(self):
        links = self.links
        not_domain_links = self.spyder.check_not_domain(self.base_url, links)
        self.assertNotIn("https://www.bbc.com/news", not_domain_links)
        self.assertIn("https://www.bbc.com", not_domain_links)
        
    def test_check_ref(self):
        links = ["https://www.example.com/about"]
        target_links = {"https://www.example.com": 0}
        ref_counts = self.spyder.check_ref(links, target_links)
        self.assertEqual(ref_counts,{"https://www.example.com": 1})
        

In [10]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestSpyder)
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.677s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>